## The objective is to solve the puzzle Sudoku for any size (order), using integer programming.

In [1]:
%matplotlib notebook
import numpy as np
import cvxopt #Linear and Integer Programming Solver
import cvxopt.glpk

mainsize=3

In [2]:
## help for the integer LP solver in cvxopt
cvxopt.glpk.ilp?

In [3]:
## To help create the huge matrix
def unravel(i,j,k,size=mainsize):
    '''
    Associate a unique variable index given a 3-index (ijk) 
    '''
    n2 = size*size
    assert(i>=0 and i<n2)
    assert(j>=0 and i<n2)
    assert(k>=0 and i<n2)
    
    return(k+ j*n2+ i*n2*n2)


""" not used, for reference only
def ravel(l,size=3):
    n2=size*size
    assert (l>=0 and l < n2*n2*n2)
    i = l // (n2*n2)
    j = (l % (n2*n2)) // n2
    k = l - i*n2*n2 - j*n2
    return ((i,j,k))
"""

' not used, for reference only\ndef ravel(l,size=3):\n    n2=size*size\n    assert (l>=0 and l < n2*n2*n2)\n    i = l // (n2*n2)\n    j = (l % (n2*n2)) // n2\n    k = l - i*n2*n2 - j*n2\n    return ((i,j,k))\n'

In [4]:
#Create constraint matrix A
size=3
n2=size*size
A=np.zeros((4*n2*n2,n2*n2*n2))
A.shape

(324, 729)

In [5]:
#Initializing constraint counter c
c=0

## First, the line constraint
## Only one number per line (each number can only appear once in a row)

In [6]:
for k in range(n2): ## for all numbers
    for j in range(n2): ## for all columns
        for i in range(n2): 
            A[c,unravel(i,j,k)] = 1 ## only one number k on line i
        c += 1 #Move to the next constraint

## Second, the column constraint. 
## Only one number per column (each number can only appear once in a column)

In [7]:
for k in range(n2): ## for all numbers
    for i in range(n2): ## for all rows
        for j in range(n2): 
            A[c,unravel(i,j,k)] = 1 ## only one number k on column j
        c += 1 #Move to the next constraint   

## Third, the single number located at any location (i,j) constraint
## We cannot have more than 1 number in the same location (i,j)
## Only one number per square (i,j)

In [8]:
for i in range(n2): ## for all rows
    for j in range(n2): ## for all columns
        for k in range(n2): 
            A[c,unravel(i,j,k)] = 1 ## only one number in each square (i,j) at a time
        c += 1 #Move to the next constraint   

## Fourth, the 3x3 subsquares constraint 
## Only one number per 3x3 block (each number can only appear once in a 3x3 block)
## We have in total 9 3x3 blocks and we have to apply this constraint to all of them

In [9]:
for k in range(n2):
    #First 3x3 block
    for i in range(0,3):
        for j in range(0,3):
            A[c,unravel(i,j,k)] = 1 ## only one number k on first 3x3 block
    c += 1 #Move to the next constraint         

    #Second 3x3 block
    for i in range(0,3):
        for j in range(3,6):
            A[c,unravel(i,j,k)] = 1 ## only one number k on second 3x3 block
    c += 1 #Move to the next constraint      

    #Third 3x3 block
    for i in range(0,3):
        for j in range(6,9):
            A[c,unravel(i,j,k)] = 1 ## only one number k on third 3x3 block
    c += 1 #Move to the next constraint   

    #Fourth 3x3 block
    for i in range(3,6):
        for j in range(0,3):
            A[c,unravel(i,j,k)] = 1 ## only one number k on fourth 3x3 block
    c += 1 #Move to the next constraint          

    #Fifth 3x3 block
    for i in range(3,6):
        for j in range(3,6):
            A[c,unravel(i,j,k)] = 1 ## only one number k on fifth 3x3 block
    c += 1 #Move to the next constraint   

    #Sixth 3x3 block
    for i in range(3,6):
        for j in range(6,9):
            A[c,unravel(i,j,k)] = 1 ## only one number k on sixth 3x3 block
    c += 1 #Move to the next constraint 

    #Seventh 3x3 block
    for i in range(6,9):
        for j in range(0,3):
            A[c,unravel(i,j,k)] = 1 ## only one number k on seventh 3x3 block
    c += 1 #Move to the next constraint          

    #Eighth 3x3 block
    for i in range(6,9):
        for j in range(3,6):
            A[c,unravel(i,j,k)] = 1 ## only one number k on eighth 3x3 block
    c += 1 #Move to the next constraint    

    #Nineth 3x3 block
    for i in range(6,9):
        for j in range(6,9):
            A[c,unravel(i,j,k)] = 1 ## only one number k on nineth 3x3 block
    c += 1 #Move to the next constraint     

In [10]:
print("Total number of constraints=",c)

def testA(A,c,size=3):
    n2 = size*size
    for n in range(c):
        if (np.sum(A[n,])!=n2):
            print("error on line", n)
            break
    print("All constraints OK")
    return

testA(A,c)

Total number of constraints= 324
All constraints OK


In [11]:
A.shape

(324, 729)

## Saving a copy of the base Constraint Matrix for later use in the hard Sudoku test

In [12]:
A2 = A

## Simple Sudoku

In [13]:
K = np.array([
[0,8,0, 9,0,1, 0,5,0],
[0,0,2, 6,8,7, 3,0,0],
[0,0,3, 0,0,0, 6,0,0],
[3,9,0, 0,0,0, 0,6,5],
[6,0,0, 4,7,5, 0,0,3],
[5,7,0, 0,0,0, 0,8,4],
[0,0,9, 0,0,0, 8,0,0],
[0,0,5, 1,2,4, 9,0,0],
[0,4,0, 8,0,3, 0,2,0]])

## Define given number constraints for Simple Sudoku

In [14]:
for i in range(n2):
    for j in range(n2):
        k = K[i,j]
        if (k>0):
            newrow=np.zeros(n2*n2*n2) #Initialize new constraint
            newrow[unravel(i,j,k-1)]=1 #Create the given number constraint 
            A=np.vstack((A,newrow)) #Add it to the constraint matrix
            
print("A.shape=", A.shape)

A.shape= (359, 729)


## Solving using cvxopt

In [15]:
from cvxopt import matrix

b=matrix(np.ones(A.shape[0])) ## set partition
c=matrix(np.zeros(A.shape[1])) ## zero cost
G=matrix(np.zeros(A.shape))
h=matrix(np.zeros(A.shape[0]))
binary=np.array(range(A.shape[1]))
I=set(binary)
B=set(range(A.shape[1]))
Aeq = matrix(A)
(status, solution) = cvxopt.glpk.ilp(c=c,G=G,h=h,A=Aeq,b=b,B=set(range(A.shape[1])))

In [16]:
#Checking solution status
status

'optimal'

## Print solution

In [17]:

def printsol(sol):
    sep3="+-----+-----+-----+"
    for i in range(n2):
        if (i%3 == 0):
            print(sep3)
        print("|",end='')
        for j in range(n2):
            for k in range(n2):
                if (sol[unravel(i,j,k)]==1):
                    print(k+1,end='')
            if (j%3 ==2):
                print("|",end='')
            else:
                print(" ",end='')
        print("")
    print(sep3)
        
printsol(solution)
          

+-----+-----+-----+
|7 8 6|9 3 1|4 5 2|
|4 5 2|6 8 7|3 1 9|
|9 1 3|5 4 2|6 7 8|
+-----+-----+-----+
|3 9 4|2 1 8|7 6 5|
|6 2 8|4 7 5|1 9 3|
|5 7 1|3 6 9|2 8 4|
+-----+-----+-----+
|2 3 9|7 5 6|8 4 1|
|8 6 5|1 2 4|9 3 7|
|1 4 7|8 9 3|5 2 6|
+-----+-----+-----+


## Hard Sudoku

In [18]:
K = np.array([
[7,0,0, 0,0,0, 4,0,0],
[0,2,0, 0,7,0, 0,8,0],
[0,0,3, 0,0,8, 0,0,9],
[0,0,0, 5,0,0, 3,0,0],
[0,6,0, 0,2,0, 0,9,0],
[0,0,1, 0,0,7, 0,0,6],
[0,0,0, 3,0,0, 9,0,0],
[0,3,0, 0,4,0, 0,6,0],
[0,0,9, 0,0,1, 0,0,5]])

## Define given number constraints for Hard Sudoku

In [19]:
for i in range(n2):
    for j in range(n2):
        k = K[i,j]
        if (k>0):
            newrow=np.zeros(n2*n2*n2) #Initialize new constraint
            newrow[unravel(i,j,k-1)]=1 #Create the given number constraint 
            A2=np.vstack((A2,newrow)) #Add it to the constraint matrix
            
print("A2.shape=", A2.shape)

A2.shape= (348, 729)


## Solving using cvxopt

In [20]:
from cvxopt import matrix

b=matrix(np.ones(A2.shape[0])) ## set partition
c=matrix(np.zeros(A2.shape[1])) ## zero cost
G=matrix(np.zeros(A2.shape))
h=matrix(np.zeros(A2.shape[0]))
binary=np.array(range(A2.shape[1]))
I=set(binary)
B=set(range(A2.shape[1]))
Aeq = matrix(A2)
(status, solution) = cvxopt.glpk.ilp(c=c,G=G,h=h,A=Aeq,b=b,B=set(range(A2.shape[1])))

In [21]:
#Checking solution status
status

'optimal'

## Print solution

In [22]:
def printsol(sol):
    sep3="+-----+-----+-----+"
    for i in range(n2):
        if (i%3 == 0):
            print(sep3)
        print("|",end='')
        for j in range(n2):
            for k in range(n2):
                if (sol[unravel(i,j,k)]==1):
                    print(k+1,end='')
            if (j%3 ==2):
                print("|",end='')
            else:
                print(" ",end='')
        print("")
    print(sep3)
        
printsol(solution)
          

+-----+-----+-----+
|7 9 8|6 3 5|4 2 1|
|1 2 6|9 7 4|5 8 3|
|4 5 3|2 1 8|6 7 9|
+-----+-----+-----+
|9 7 2|5 8 6|3 1 4|
|5 6 4|1 2 3|8 9 7|
|3 8 1|4 9 7|2 5 6|
+-----+-----+-----+
|6 1 7|3 5 2|9 4 8|
|8 3 5|7 4 9|1 6 2|
|2 4 9|8 6 1|7 3 5|
+-----+-----+-----+


## Now let's solve a Sudoku 4x4

In [23]:
mainsize=4

In [24]:
## To help create the huge matrix
def unravel(i,j,k,size=mainsize):
    '''
    Associate a unique variable index given a 3-index (ijk) 
    '''
    n2 = size*size
    assert(i>=0 and i<n2)
    assert(j>=0 and i<n2)
    assert(k>=0 and i<n2)
    
    return(k+ j*n2+ i*n2*n2)


""" not used, for reference only
def ravel(l,size=3):
    n2=size*size
    assert (l>=0 and l < n2*n2*n2)
    i = l // (n2*n2)
    j = (l % (n2*n2)) // n2
    k = l - i*n2*n2 - j*n2
    return ((i,j,k))
"""

' not used, for reference only\ndef ravel(l,size=3):\n    n2=size*size\n    assert (l>=0 and l < n2*n2*n2)\n    i = l // (n2*n2)\n    j = (l % (n2*n2)) // n2\n    k = l - i*n2*n2 - j*n2\n    return ((i,j,k))\n'

In [25]:
#Create constraint matrix A
size=4
n2=size*size
A=np.zeros((4*n2*n2,n2*n2*n2))
A.shape

(1024, 4096)

In [26]:
#Initializing constraint counter c
c=0

## First, the line constraint
## Only one number per line (each number can only appear once in a row)

In [27]:
for k in range(n2): ## for all numbers
    for j in range(n2): ## for all columns
        for i in range(n2): 
            A[c,unravel(i,j,k)] = 1 ## only one number k on line i
        c += 1 #Move to the next constraint

## Second, the column constraint. 
## Only one number per column (each number can only appear once in a column)

In [28]:
for k in range(n2): ## for all numbers
    for i in range(n2): ## for all rows
        for j in range(n2): 
            A[c,unravel(i,j,k)] = 1 ## only one number k on column j
        c += 1 #Move to the next constraint   

## Third, the single number located at any location (i,j) constraint
## We cannot have more than 1 number in the same location (i,j)
## Only one number per square (i,j)

In [29]:
for i in range(n2): ## for all rows
    for j in range(n2): ## for all columns
        for k in range(n2): 
            A[c,unravel(i,j,k)] = 1 ## only one number in each square (i,j) at a time
        c += 1 #Move to the next constraint   

## Fourth, the 4x4 subsquares constraint 
## Only one number per 4x4 block (each number can only appear once in a 4x4 block)
## We have in total 16 4x4 blocks and we have to apply this constraint to all of them

In [30]:
for k in range(n2):
    #First 4x4 block
    for i in range(0,4):
        for j in range(0,4):
            A[c,unravel(i,j,k)] = 1 ## only one number k on first 4x4 block
    c += 1 #Move to the next constraint         

    #Second 4x4 block
    for i in range(0,4):
        for j in range(4,8):
            A[c,unravel(i,j,k)] = 1 ## only one number k on second 4x4 block
    c += 1 #Move to the next constraint      

    #Third 4x4 block
    for i in range(0,4):
        for j in range(8,12):
            A[c,unravel(i,j,k)] = 1 ## only one number k on third 4x4 block
    c += 1 #Move to the next constraint   

    #Fourth 4x4 block
    for i in range(0,4):
        for j in range(12,16):
            A[c,unravel(i,j,k)] = 1 ## only one number k on fourth 4x4 block
    c += 1 #Move to the next constraint          

    #Fifth 4x4 block
    for i in range(4,8):
        for j in range(0,4):
            A[c,unravel(i,j,k)] = 1 ## only one number k on fifth 4x4 block
    c += 1 #Move to the next constraint   

    #Sixth 4x4 block
    for i in range(4,8):
        for j in range(4,8):
            A[c,unravel(i,j,k)] = 1 ## only one number k on sixth 4x4 block
    c += 1 #Move to the next constraint 

    #Seventh 4x4 block
    for i in range(4,8):
        for j in range(8,12):
            A[c,unravel(i,j,k)] = 1 ## only one number k on seventh 4x4 block
    c += 1 #Move to the next constraint          

    #Eighth 4x4 block
    for i in range(4,8):
        for j in range(12,16):
            A[c,unravel(i,j,k)] = 1 ## only one number k on eighth 4x4 block
    c += 1 #Move to the next constraint    

    #Nineth 4x4 block
    for i in range(8,12):
        for j in range(0,4):
            A[c,unravel(i,j,k)] = 1 ## only one number k on nineth 4x4 block
    c += 1 #Move to the next constraint   

    #Tenth 4x4 block
    for i in range(8,12):
        for j in range(4,8):
            A[c,unravel(i,j,k)] = 1 ## only one number k on tenth 4x4 block
    c += 1 #Move to the next constraint 

    #Eleventh 4x4 block
    for i in range(8,12):
        for j in range(8,12):
            A[c,unravel(i,j,k)] = 1 ## only one number k on eleventh 4x4 block
    c += 1 #Move to the next constraint          

    #Twelfth 4x4 block
    for i in range(8,12):
        for j in range(12,16):
            A[c,unravel(i,j,k)] = 1 ## only one number k on twelfth 4x4 block
    c += 1 #Move to the next constraint     
    
    #Thirteenth 4x4 block
    for i in range(12,16):
        for j in range(0,4):
            A[c,unravel(i,j,k)] = 1 ## only one number k on thirteenth 4x4 block
    c += 1 #Move to the next constraint   

    #Fourteenth 4x4 block
    for i in range(12,16):
        for j in range(4,8):
            A[c,unravel(i,j,k)] = 1 ## only one number k on fourteenth 4x4 block
    c += 1 #Move to the next constraint 

    #Fifteenth 4x4 block
    for i in range(12,16):
        for j in range(8,12):
            A[c,unravel(i,j,k)] = 1 ## only one number k on fifteenth 4x4 block
    c += 1 #Move to the next constraint          

    #Sixteenth 4x4 block
    for i in range(12,16):
        for j in range(12,16):
            A[c,unravel(i,j,k)] = 1 ## only one number k on sixteenth 4x4 block
    c += 1 #Move to the next constraint      

In [31]:
print("Total number of constraints=",c)

def testA(A,c,size=3):
    n2 = size*size
    for n in range(c):
        if (np.sum(A[n,])!=n2):
            print("error on line", n)
            break
    print("All constraints OK")
    return

testA(A,c)

Total number of constraints= 1024
error on line 0
All constraints OK


In [32]:
A.shape

(1024, 4096)

## To define the inital number constraints, we sum 1 to each number so that we can represent a '0' as '1' and therefore not mixed it up with the blank spaces represented strategically as '0'

In [33]:
K = np.array([
[9,16,0,13, 0,0,0,0, 0,11,0,0, 0,0,0,7],
[0,0,0,11, 0,0,0,16, 0,0,0,12, 8,5,14,0],
[12,0,5,0, 0,0,14,7, 0,8,0,0, 1,0,6,0],
[2,0,0,0, 0,0,0,1, 4,0,10,3, 0,0,0,0],
[0,0,0,0, 0,2,16,14, 0,4,1,0, 0,15,8,5],
[0,2,0,7, 0,0,0,13, 0,12,0,0, 11,0,4,0],
[0,13,0,14, 0,0,7,4, 0,6,0,0, 10,3,0,0],
[10,0,4,5, 15,0,3,0, 0,0,8,14, 0,0,0,0],
[0,0,0,0, 6,8,0,0, 0,9,0,13, 4,1,0,11],
[0,0,15,3, 0,0,5,0, 8,2,0,0, 16,0,7,0],
[0,6,0,4, 0,0,9,0, 10,0,0,0, 15,0,13,0],
[8,1,7,0, 0,13,10,0, 14,15,4,0, 0,0,0,0],
[0,0,0,0, 14,15,0,5, 1,0,0,0, 0,0,0,3],
[0,8,0,9, 0,0,13,0, 5,3,0,0, 0,12,0,6],
[0,3,10,15, 12,0,0,0, 6,0,0,0, 5,0,0,0],
[7,0,0,0, 0,0,8,0, 0,0,0,0, 2,0,9,4]])

## Define given number constraints of 4x4 hard Sudoku

In [34]:
for i in range(n2):
    for j in range(n2):
        k = K[i,j]
        if (k>0):
            newrow=np.zeros(n2*n2*n2) #Initialize new constraint
            newrow[unravel(i,j,k-1)]=1 #Create the given number constraint 
            A=np.vstack((A,newrow)) #Add it to the constraint matrix
            
print("A.shape=", A.shape)

A.shape= (1126, 4096)


## Solving using cvxopt

In [35]:
from cvxopt import matrix

b=matrix(np.ones(A.shape[0])) ## set partition
c=matrix(np.zeros(A.shape[1])) ## zero cost
G=matrix(np.zeros(A.shape))
h=matrix(np.zeros(A.shape[0]))
binary=np.array(range(A.shape[1]))
I=set(binary)
B=set(range(A.shape[1]))
Aeq = matrix(A)
(status, solution) = cvxopt.glpk.ilp(c=c,G=G,h=h,A=Aeq,b=b,B=set(range(A.shape[1])))

In [36]:
#Checking solution status
status

'optimal'

## Print solution

In [37]:
def printsol(sol):
    sep4="+-------+-------+-------+-------+"
    for i in range(n2):
        if (i%4 == 0):
            print(sep4)
        print("|",end='')
        for j in range(n2):
            for k in range(n2):
                if (sol[unravel(i,j,k)]==1):
                    if k<10:
                        print(k,end='')
                    elif k==10:
                        print("A",end='')
                    elif k==11:
                        print("B",end='')     
                    elif k==12:
                        print("C",end='')     
                    elif k==13:
                        print("D",end='')       
                    elif k==14:
                        print("E",end='')       
                    elif k==15:
                        print("F",end='')                                     
            if (j%4 ==3):
                print("|",end='')
            else:
                print(" ",end='')
        print("")
    print(sep4)
        
printsol(solution)
          

+-------+-------+-------+-------+
|8 F 0 C|1 B 5 7|E A D 4|2 3 9 6|
|3 6 2 A|8 9 E F|1 0 5 B|7 4 D C|
|B E 4 9|2 3 D 6|F 7 C 8|0 A 5 1|
|1 D 5 7|C 4 A 0|3 6 9 2|B 8 F E|
+-------+-------+-------+-------+
|2 B 8 5|9 1 F D|A 3 0 6|C E 7 4|
|E 1 F 6|7 8 0 C|2 B 4 9|A 5 3 D|
|0 C 7 D|4 A 6 3|8 5 1 E|9 2 B F|
|9 A 3 4|E 5 2 B|C F 7 D|8 6 1 0|
+-------+-------+-------+-------+
|4 9 D 1|5 7 B E|6 8 F C|3 0 2 A|
|C 8 E 2|3 0 4 A|7 1 B 5|F D 6 9|
|F 5 A 3|6 D 8 1|9 4 2 0|E 7 C B|
|7 0 6 B|F C 9 2|D E 3 A|5 1 4 8|
+-------+-------+-------+-------+
|5 3 B F|D E 1 4|0 9 8 7|6 C A 2|
|A 7 1 8|0 F C 9|4 2 6 3|D B E 5|
|D 2 9 E|B 6 3 8|5 C A 1|4 F 0 7|
|6 4 C 0|A 2 7 5|B D E F|1 9 8 3|
+-------+-------+-------+-------+


## End Sudoku